## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-12-08-48-00 +0000,wsj,"Eurozone Government Bond Yields Edge Higher, T...",https://www.wsj.com/finance/investing/jgbs-mix...
1,2025-12-12-08-42-00 +0000,wsj,Gold Poised for Weekly Gains as Traders Hope f...,https://www.wsj.com/finance/currencies/gold-st...
2,2025-12-12-08-39-57 +0000,wsj,Stock Market Today: Dow Futures Rise; Nasdaq F...,https://www.wsj.com/livecoverage/stock-market-...
3,2025-12-12-08-39-18 +0000,bbc,Workers go on strike at Guinness canning site,https://www.bbc.com/news/articles/cdrnzrg6np0o...
4,2025-12-12-08-39-04 +0000,bbc,UK economy shrank unexpectedly by 0.1% in October,https://www.bbc.com/news/articles/cwyp7v7r28yo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,42
65,new,19
249,oil,16
47,ai,15
367,time,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
74,2025-12-12-01-46-00 +0000,wsj,The Indiana Senate has rejected a redistrictin...,https://www.wsj.com/politics/policy/indiana-re...,106
54,2025-12-12-02-38-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...,105
190,2025-12-11-20-14-51 +0000,wapo,U.S. seizes ‘very large’ oil tanker off Venezu...,https://www.washingtonpost.com/world/2025/12/1...,91
115,2025-12-11-23-35-16 +0000,nyt,What to Know About Trump’s Seizure of an Oil T...,https://www.nytimes.com/2025/12/11/us/politics...,85
285,2025-12-11-14-17-43 +0000,nyt,Trump Gets to Pick a New Fed Chair but a Divid...,https://www.nytimes.com/2025/12/11/business/fe...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
74,106,2025-12-12-01-46-00 +0000,wsj,The Indiana Senate has rejected a redistrictin...,https://www.wsj.com/politics/policy/indiana-re...
242,56,2025-12-11-17-27-39 +0000,nypost,Disney to let users make AI videos featuring b...,https://nypost.com/2025/12/11/business/disney-...
125,55,2025-12-11-22-59-25 +0000,nyt,Nobel Peace Prize Winner Machado Says U.S. Hel...,https://www.nytimes.com/2025/12/11/world/ameri...
190,49,2025-12-11-20-14-51 +0000,wapo,U.S. seizes ‘very large’ oil tanker off Venezu...,https://www.washingtonpost.com/world/2025/12/1...
235,45,2025-12-11-17-51-00 +0000,wsj,The U.S. trade deficit narrowed to its lowest ...,https://www.wsj.com/economy/trade/u-s-trade-de...
247,44,2025-12-11-17-00-00 +0000,wsj,Two progressives competing in the Democratic p...,https://www.wsj.com/politics/elections/a-bible...
102,42,2025-12-12-00-02-02 +0000,bbc,Reddit launches High Court challenge to Austra...,https://www.bbc.com/news/articles/clykk2yrl9ko...
202,41,2025-12-11-19-38-28 +0000,cbc,National museum CEO leaves role after report a...,https://www.cbc.ca/news/politics/pier-21-ceo-d...
267,38,2025-12-11-15-40-06 +0000,nypost,CNN denies banning Stephen Miller after top WH...,https://nypost.com/2025/12/11/business/cnn-den...
321,36,2025-12-11-11-48-15 +0000,nypost,Heart-stopping moment skydiver’s parachute get...,https://nypost.com/2025/12/11/world-news/heart...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
